# Biome Level statistics for model: GS1_MeanScaler


In [12]:
# import the libraries
import ee
import pandas as pd
import os
import numpy as np
import random
from random import sample
import itertools 
import geopandas as gpd
from sklearn.metrics import r2_score
from termcolor import colored # this is allocate colour and fonts type for the print title and text
from IPython.display import display, HTML

In [13]:
#set the working directory of local drive for Grid search result table loading
# os.getcwd()

In [14]:
# initialize the earth engine API
ee.Initialize()

## 1 Load the required composites and images

In [15]:
# load the basic maps that needed for the analysis
# load the carbon concentration map
carbonConcentration = ee.Image("users/leonidmoore/ForestBiomass/Biome_level_Wood_Carbon_Conentration_Map")
# load the root shoot ratio map
rootShootRatio = ee.Image("users/leonidmoore/ForestBiomass/Root_shoot_ratio_Map").unmask()
# load the two composites tha will be used in the analysis
compositeImage =ee.Image("users/leonidmoore/ForestBiomass/20200915_Forest_Biomass_Predictors_Image")
compositeImageNew = ee.Image("projects/crowtherlab/Composite/CrowtherLab_Composite_30ArcSec")
# load the biome layer 
biomeLayer = compositeImage.select("WWF_Biome")
# define a pixel area layer with unit km2
pixelAreaMap = ee.Image.pixelArea().divide(10000);
# define the boundary geography reference
unboundedGeo = ee.Geometry.Polygon([-180, 88, 0, 88, 180, 88, 180, -88, 0, -88, -180, -88], None, False)

## 2 Load the biomass density maps

In [16]:
# load the carbon density layers
potentialDensity = ee.Image("users/leonidmoore/ForestBiomass/GroundSourcedModel/EnsambledMaps/Predicted_GS1_MeanScaler_Potential_density_Ensambled_Mean_20230427").unmask()
presentDensity = ee.Image("users/leonidmoore/ForestBiomass/GroundSourcedModel/EnsambledMaps/Predicted_GS1_MeanScaler_Present_density_Ensambled_Mean_20230427").unmask()

# define the standard projection
stdProj = potentialDensity.projection();

## 3 Adjust the present and potential density maps

In [17]:
# load the present and potential forest cover
presentForestCover = compositeImage.select('PresentTreeCover').unmask() # uniform with potential in the  0-1 scale
potentialCoverAdjusted = ee.Image("users/leonidmoore/ForestBiomass/Bastin_et_al_2019_Potential_Forest_Cover_Adjusted").unmask().rename('PotentialForestCover')
# define the present and potential forest cover masks
presentMask= presentForestCover.gt(0)
potentialMask= potentialCoverAdjusted.gt(0)

# check the difference of the two density maps
potentialHigher = potentialDensity.multiply(pixelAreaMap).subtract(presentDensity.multiply(pixelAreaMap)).gte(0)
potentialLower = potentialDensity.multiply(pixelAreaMap).subtract(presentDensity.multiply(pixelAreaMap)).lt(0)
# replace the lower potential value by present biomass density value
agbPotentialDensity = presentDensity.multiply(potentialLower).add(potentialDensity.multiply(potentialHigher))
# add the root biomass to the AGB to get TGB
tgbPotentialDensity = agbPotentialDensity.multiply(rootShootRatio).add(agbPotentialDensity)
tgbPresentDensity = presentDensity.multiply(rootShootRatio).add(presentDensity)

## 4 Partioning the potential cover into different landuse types

In [18]:
# Load all the landuse type layers
croplandOrg = ee.Image("users/leonidmoore/ForestBiomass/HYDE31/cropland_Percent").rename('cropland').divide(100).reproject(crs=stdProj);
grazingOrg = ee.Image("users/leonidmoore/ForestBiomass/HYDE31/grazing_Percent").rename('grazing').divide(100).reproject(crs=stdProj);
pastureOrg = ee.Image("users/leonidmoore/ForestBiomass/HYDE31/pasture_Percent").rename('pasture').divide(100).reproject(crs=stdProj);
rangelandOrg = ee.Image("users/leonidmoore/ForestBiomass/HYDE31/rangeland_Percent").rename('rangeland').divide(100).reproject(crs=stdProj);
urbanOrg = compositeImage.select(['LandCoverClass_Urban_Builtup']).divide(100).unmask().reproject(crs=stdProj);
snowIceOrg = compositeImageNew.select(['ConsensusLandCoverClass_Snow_Ice']).divide(100).unmask().reproject(crs=stdProj);
openWaterOrg = compositeImageNew.select(['ConsensusLandCoverClass_Open_Water']).divide(100).unmask().reproject(crs=stdProj);
# define the total landcover types
sumCover = presentForestCover.add(pastureOrg).add(rangelandOrg).add(croplandOrg).add(urbanOrg).add(openWaterOrg).add(snowIceOrg);
oneSubtract = ee.Image(1).subtract(sumCover);
freeland = oneSubtract.multiply(oneSubtract.gte(0));
# get the scale ratio for pixels with sumCover larger than 1
scaleRatio = ee.Image(1).subtract(presentForestCover).divide(sumCover.subtract(presentForestCover)).multiply(oneSubtract.lt(0));
# get the ratio of these three disturbed maps
pasture = pastureOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(pastureOrg.multiply(oneSubtract.gte(0))).unmask();
rangeland = rangelandOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(rangelandOrg.multiply(oneSubtract.gte(0))).unmask();
cropland = croplandOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(croplandOrg.multiply(oneSubtract.gte(0))).unmask();
urban = urbanOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(urbanOrg.multiply(oneSubtract.gte(0))).unmask();
openWater = openWaterOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(openWaterOrg.multiply(oneSubtract.gte(0))).unmask();
snowIce = snowIceOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(snowIceOrg.multiply(oneSubtract.gte(0))).unmask();
sumTT = presentForestCover.add(pasture).add(rangeland).add(cropland).add(urban).add(freeland).add(openWater).add(snowIce).unmask();

effectivePotentialMask = freeland.add(rangeland).add(pasture).add(cropland).add(urban).gt(0);
# there are some pixels without any landcover survived but with open water and ice and snow. here we mask these pixels out
sumlandCover = pastureOrg.add(rangelandOrg).add(croplandOrg).add(urbanOrg).add(freeland);
restorationMap = potentialCoverAdjusted.subtract(presentForestCover).mask(effectivePotentialMask).unmask();

# sum all these scaled layersv
scaledSum = pasture.add(rangeland).add(cropland).add(urban).add(freeland);
potentialCoverFinal = restorationMap.add(presentForestCover);
# allocate the potential equally to each layer
freelandPotentialCover = freeland.divide(scaledSum).multiply(restorationMap).unmask();
rangelandPotentialCover = rangeland.divide(scaledSum).multiply(restorationMap).unmask();
pasturePotentialCover = pasture.divide(scaledSum).multiply(restorationMap).unmask();
croplandPotentialCover = cropland.divide(scaledSum).multiply(restorationMap).unmask();
urbanPotentialCover = urban.divide(scaledSum).multiply(restorationMap).unmask();
#  allocate the freeland potential in pixels with forest cover larger than 10% to conservation potential
freelandForConsevation = freelandPotentialCover.multiply(presentForestCover.gte(0.1)).unmask();
maximumPotentialCover = freelandForConsevation.add(presentForestCover);
# calucate the reall freeland outside of forest
freelandLeftMap = freelandPotentialCover.subtract(freelandForConsevation).unmask()# the left positive pixels are real freeland pixels

## 5 Partioning the biomass potential into different landuse types

In [19]:
# calculate the existing carbon, present potential carbon and absolute potential carbon in forests
absoluteImage1 = tgbPresentDensity.multiply(pixelAreaMap).multiply(presentMask).divide(1000000000).rename('Present')
absoluteImage3 = tgbPotentialDensity.multiply(pixelAreaMap).multiply(potentialCoverFinal.gt(0)).divide(1000000000).rename('AbsolutePotential')
# get the sum of the potential covers
potentialCoverSum = freelandLeftMap.add(rangelandPotentialCover).add(pasturePotentialCover).add(croplandPotentialCover).add(urbanPotentialCover)

trueRestorationPotential = absoluteImage3.subtract(absoluteImage1).multiply(1000000000)
ratioPotentialBiomassDensity = absoluteImage1.multiply(potentialCoverFinal.divide(presentForestCover))
#  get the real for conservation potential
realDensityIncreased = absoluteImage3.subtract(absoluteImage1).mask(absoluteImage3.subtract(ratioPotentialBiomassDensity).gt(0)).unmask()
realDensityNotIncreased = absoluteImage3.subtract(absoluteImage1).mask(absoluteImage3.subtract(ratioPotentialBiomassDensity).lte(0)).unmask()
trueReforestationPotential = realDensityNotIncreased.add(realDensityIncreased.multiply(ee.Image(1).subtract(presentForestCover.divide(potentialCoverFinal))))

conservationPotentialPart1 = realDensityIncreased.multiply(presentForestCover.add(freelandForConsevation).divide(potentialCoverFinal))
conservationPotentialPart2 = realDensityNotIncreased.multiply(freelandForConsevation.divide(potentialCoverFinal.subtract(presentForestCover)))

# calculate the part of the potential inside the forest cover which was allocate to conservation potential.
freelandForConservation1 = realDensityIncreased.multiply(freelandForConsevation.divide(potentialCoverFinal))
freelandForConservation = freelandForConservation1.add(conservationPotentialPart2).rename('FreeToConservation')

absoluteImage2 = conservationPotentialPart1.add(conservationPotentialPart2).add(absoluteImage1).rename('PresentPotential')

trueReforestationPotential = absoluteImage3.subtract(absoluteImage2)

absoluteImage4 = trueReforestationPotential.multiply(freelandLeftMap.divide(potentialCoverSum)).rename('FreelandPotential')
absoluteImage5 = trueReforestationPotential.multiply(rangelandPotentialCover.divide(potentialCoverSum)).rename('RangelandPotential')
absoluteImage6 = trueReforestationPotential.multiply(pasturePotentialCover.divide(potentialCoverSum)).rename('PasturePotential')
absoluteImage7 = trueReforestationPotential.multiply(croplandPotentialCover.divide(potentialCoverSum)).rename('CroplandPotential')
absoluteImage8 = trueReforestationPotential.multiply(urbanPotentialCover.divide(potentialCoverSum)).rename('UrbanPotential')

## Calculate the potential numbers and write into local folder

### TGB

In [20]:
# Stack the absolute biomass layers into an Image.
absPotentialImage = absoluteImage1.addBands(absoluteImage2).addBands(absoluteImage3).addBands(absoluteImage4).addBands(absoluteImage5).addBands(absoluteImage6).addBands(absoluteImage7).addBands(absoluteImage8).addBands(freelandForConservation)
# define the function to do the biome level statistics which could be applied by map      
def biomeLevelStat(biome):
    biomeMask = biomeLayer.eq(ee.Number(biome))
    masked_img = absPotentialImage.mask(biomeMask)
    output = masked_img.reduceRegion(reducer= ee.Reducer.sum(),
                                     geometry= unboundedGeo,
                                     crs='EPSG:4326',
                                     crsTransform=[0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                     maxPixels= 1e13)
    return output#.getInfo().get('Present')


biomeList = ee.List([1,2,3,4,5,6,7,8,9,10,11,12,13,14])
statisticTable = biomeList.map(biomeLevelStat).getInfo()
# transform into data frame
outputTable = pd.DataFrame(statisticTable,columns =['Present','PresentPotential','AbsolutePotential','FreelandPotential','RangelandPotential','PasturePotential','CroplandPotential','UrbanPotential','FreeToConservation'])#.round(1)
outputTable.loc['sum'] = outputTable.sum() 
outputTable.to_csv('Data/BiomeLevelStatistics/StatisticsForModels/GS1_MeanScaler_Biome_Level_Statistics.csv',header=True,mode='w+')
# display the output of the carbon partitioning
print(colored('The biomass partition results in biome: \n', 'blue', attrs=['bold']))
outputTable.head(15)

The biomass partition results in biome: 



,Present,PresentPotential,AbsolutePotential,FreelandPotential,RangelandPotential,PasturePotential,CroplandPotential,UrbanPotential,FreeToConservation
0,233.543439,263.431798,289.648569,4.875155,0.077979,9.224247,11.707135,0.332255,9.189539
1,7.873063,10.775928,18.050761,1.764999,0.017055,1.992831,3.458733,0.041215,0.996328
2,3.074780,4.517766,5.602595,0.382421,0.017910,0.397145,0.283533,0.003821,0.497629
3,40.546435,50.628121,68.488364,4.107664,0.021477,5.243814,7.971122,0.516167,4.321384
4,19.024499,23.389593,26.261260,1.321708,0.034382,0.862857,0.601835,0.050885,2.439337
5,29.181260,31.934119,34.288844,2.019667,0.000764,0.155589,0.167467,0.011238,2.468113
6,47.409114,66.911867,96.427826,5.442853,9.859583,8.454343,5.692787,0.066393,5.431991
7,3.654017,4.402528,16.695203,2.222140,3.636817,1.713386,4.649400,0.070932,0.234091
8,1.732141,2.394504,3.362576,0.259703,0.362871,0.211430,0.126004,0.008063,0.166564
9,3.527103,4.987713,8.029850,0.765829,1.058465,0.600219,0.611165,0.006460,0.315194


In [21]:
# If you got the error 'EEException: Too many concurrent aggregations.', please re-run this chunck of code again.

### PGB

In [23]:
deadWoodLitterRatio = ee.Image("users/leonidmoore/ForestBiomass/DeadWoodLitter/DeadWood_Litter_Ratio_Map").unmask()

# Stack the absolute biomass layers into an Image.
absPotentialImage = absoluteImage1.addBands(absoluteImage2).addBands(absoluteImage3).addBands(absoluteImage4).addBands(absoluteImage5).addBands(absoluteImage6).addBands(absoluteImage7).addBands(absoluteImage8).addBands(freelandForConservation)
# define the function to do the biome level statistics which could be applied by map      
def biomeLevelStat(biome):
    biomeMask = biomeLayer.eq(ee.Number(biome))
    masked_img = absPotentialImage.mask(biomeMask).multiply(deadWoodLitterRatio)
    output = masked_img.reduceRegion(reducer= ee.Reducer.sum(),
                                     geometry= unboundedGeo,
                                     crs='EPSG:4326',
                                     crsTransform=[0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                     maxPixels= 1e13)
    return output#.getInfo().get('Present')


biomeList = ee.List([1,2,3,4,5,6,7,8,9,10,11,12,13,14])
statisticTable = biomeList.map(biomeLevelStat).getInfo()
# transform into data frame
outputTable = pd.DataFrame(statisticTable,columns =['Present','PresentPotential','AbsolutePotential','FreelandPotential','RangelandPotential','PasturePotential','CroplandPotential','UrbanPotential','FreeToConservation'])#.round(1)
outputTable.loc['sum'] = outputTable.sum() 
outputTable.to_csv('Data/BiomeLevelStatistics/StatisticsForModels/GS1_MeanScaler_Biome_Level_Statistics_with_Litter.csv',header=True,mode='w+')
# display the output of the carbon partitioning
print(colored('The biomass partition results in biome: \n', 'blue', attrs=['bold']))
outputTable.head(15)

The biomass partition results in biome: 



,Present,PresentPotential,AbsolutePotential,FreelandPotential,RangelandPotential,PasturePotential,CroplandPotential,UrbanPotential,FreeToConservation
0,284.933366,321.400587,353.387488,5.948162,0.095296,11.254433,14.283656,0.405353,11.212024
1,9.605110,13.146586,22.021975,2.153304,0.020888,2.431287,4.219627,0.050282,1.215511
2,3.751965,5.512981,6.836668,0.466651,0.021826,0.484504,0.346044,0.004662,0.607350
3,53.922454,67.328955,91.080607,5.462220,0.028530,6.973755,10.600712,0.686436,5.746406
4,25.301132,31.105444,34.922612,1.756750,0.045470,1.147260,0.800040,0.067648,3.243682
5,52.498388,57.450876,61.685731,3.633831,0.001295,0.279464,0.300087,0.020178,4.440411
6,57.839603,81.633313,117.643445,6.640370,12.028959,10.314521,6.945281,0.081002,6.627030
7,4.860284,5.855832,22.204200,2.955287,4.836393,2.278685,6.183669,0.094335,0.311388
8,2.113578,2.921733,4.102950,0.316886,0.442706,0.257986,0.153792,0.009847,0.203243
9,4.680865,6.621132,10.664227,1.017706,1.406935,0.797640,0.812228,0.008586,0.418640


In [97]:
# If you got the error 'EEException: Too many concurrent aggregations.', please re-run this chunck of code again.